# Load pytorch library
Select the device to train the CNN! "cuda:0" means the first GPU device.

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt

import net

net = net.Net()
print(sum([p.numel() for p in net.parameters()]))

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
net = net.to(device)

884330
cuda:0


# MNIST training and validation set augmentation

In [2]:
valid_ratio = 0.3

transform = transforms.Compose([
    # define your data augmentation here!
    # transforms.RandomRotation(degrees=30),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_valid_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
nb_train = int((1.0 - valid_ratio) * len(train_valid_dataset))
nb_valid =  int(valid_ratio * len(train_valid_dataset))
train_dataset, valid_dataset = torch.utils.data.dataset.random_split(train_valid_dataset, [nb_train, nb_valid])
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=1000, shuffle=True, num_workers=2, pin_memory=True)
validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=1000, shuffle=True, num_workers=2, pin_memory=True)

classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')

# Define the loss function and the optimizer.

In [3]:
import torch.nn as nn
import helper
import torch.optim as optim

criterion = nn.CrossEntropyLoss()

# define your optimizer and its learning rate (lr) here!
optimizer = optim.Adam(net.parameters(), lr=0.001)

learningRates = [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10]

# Train the CNN and store the best model based on the validation loss.

In [4]:
import time
import os as OO

PATH = 'C:/Users/Ryan/Desktop/Part 2/cnn'
helper.runCNN(trainloader, device, optimizer, net, criterion, validloader, PATH, 'cnn.pth', 10)

# Define the test dataset.

In [65]:
transform = transforms.Compose([
     transforms.ToTensor(),
     transforms.Normalize((0.1307,), (0.3081,))
])
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=1, pin_memory=True)

# Infer on the whole test dataset.

In [66]:
testloader = torch.utils.data.DataLoader(testset, batch_size=200, shuffle=False, num_workers=1)
net.load_state_dict(torch.load(OO.path.join(PATH, 'cnn.pth')))

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images.to(device))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.to(device)).sum().item()

print('Accuracy of the network on the 10000 test images: %.3F %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 94.550 %


# check the GPU device assigned by Google.

In [67]:
# !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
import subprocess
print(subprocess.getoutput('nvidia-smi'))

Wed Dec 16 18:53:10 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 456.71       Driver Version: 456.71       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1070   WDDM  | 00000000:0B:00.0  On |                  N/A |
| 27%   59C    P0    43W / 151W |   4147MiB /  8192MiB |      4%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU  